- in order to get analyzed but not normalized dataset.
- delete chart code to decreate running time

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm

import scipy

## load data

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)

In [4]:
pth_data = '../data/train.csv'
df_origin = pd.read_csv(pth_data)

In [5]:
df_origin

,time,height,weight,gender,bmi,age,env_temp,rh,heart_rate,stress_level,skin_temp,eda,TC,TS,Clo,Act
0,2020-10-19 14:00:00,1.64,54.0,1,20.077335,24,77.990,62.122,60.0,17.0,NaN,NaN,4.0,4.0,0.74,1.0
1,2020-10-19 14:30:00,1.64,54.0,1,20.077335,24,78.208,62.321,78.0,60.0,30.748,0.926320,4.0,4.0,0.74,1.1
2,2020-10-19 15:00:00,1.64,54.0,1,20.077335,24,78.514,62.255,79.0,46.0,31.327,1.232405,4.0,4.0,0.74,1.1
3,2020-10-19 15:30:00,1.64,54.0,1,20.077335,24,79.041,61.491,94.0,91.0,31.458,0.375005,4.0,4.0,0.74,1.0
4,2020-10-19 16:00:00,1.64,54.0,1,20.077335,24,79.435,61.071,85.0,71.0,32.085,0.081127,4.0,4.0,0.74,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2791,2020-10-28 21:30:00,1.63,50.0,1,18.818924,23,76.528,40.530,73.0,22.0,31.850,0.114876,2.0,3.0,0.50,1.0
2792,2020-10-28 22:00:00,1.63,50.0,1,18.818924,23,76.876,39.890,70.5,10.0,32.215,0.146231,2.0,3.0,0.50,1.0
2793,2020-10-28 22:30:00,1.63,50.0,1,18.818924,23,77.050,39.710,72.0,5.0,32.656,0.153838,2.0,3.0,0.50,1.0
2794,2020-10-28 23:00:00,1.63,50.0,1,18.818924,23,76.703,39.980,66.0,0.0,31.902,0.160930,2.0,3.0,0.50,1.0


In [6]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2796 entries, 0 to 2795
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   time          2796 non-null   object 
 1   height        2796 non-null   float64
 2   weight        2796 non-null   float64
 3   gender        2796 non-null   int64  
 4   bmi           2796 non-null   float64
 5   age           2796 non-null   int64  
 6   env_temp      2796 non-null   float64
 7   rh            2796 non-null   float64
 8   heart_rate    2201 non-null   float64
 9   stress_level  1848 non-null   float64
 10  skin_temp     1874 non-null   float64
 11  eda           2396 non-null   float64
 12  TC            2717 non-null   float64
 13  TS            2712 non-null   float64
 14  Clo           2737 non-null   float64
 15  Act           2718 non-null   float64
dtypes: float64(13), int64(2), object(1)
memory usage: 349.6+ KB


## data process

### choose data: TS >= 3.0

In [7]:
df_origin[df_origin[['TS']].isna().TS].shape
# some na in clo, means unrecorded data

(84, 16)

In [8]:
df_record = df_origin[df_origin['TS'].isna() ^ True]

In [9]:
df_record.shape

(2712, 16)

In [10]:
df_comfort = df_record[df_record['TC'] >= 3.0]

### shuffle

In [11]:
df_shuffle = df_comfort.sample(frac=1, random_state=1208).reset_index(drop=True)

In [12]:
X_data = df_shuffle[['height', 'weight', 'gender', 'bmi', 'age', 'rh', \
                      'heart_rate', 'stress_level', 'skin_temp', 'eda', \
                      'TC', 'TS', 'Clo', 'Act']]

In [13]:
X_cols_name = X_data.columns

In [14]:
y_data = df_shuffle['env_temp']

### imputation

In [15]:
df_record[df_record[['env_temp']].isna().env_temp]
# no na in env_temp

,time,height,weight,gender,bmi,age,env_temp,rh,heart_rate,stress_level,skin_temp,eda,TC,TS,Clo,Act


In [16]:
imp = SimpleImputer(strategy='mean')

In [17]:
X_data = pd.DataFrame(imp.fit_transform(X_data), columns=X_cols_name)

In [18]:
X_data_not_norm = pd.DataFrame(imp.fit_transform(X_data), columns=X_cols_name)

### normalize

In [19]:
X_array_normalize, X_norm = normalize(X_data, axis=0, return_norm=True)

In [20]:
X_data = pd.DataFrame(X_array_normalize, columns=X_cols_name)

In [21]:
X_data

,height,weight,gender,bmi,age,rh,heart_rate,stress_level,skin_temp,eda,TC,TS,Clo,Act
0,0.018674,0.016305,0.019889,0.018573,0.019347,0.024127,0.020630,0.010107,0.019882,0.000866,0.020182,0.018018,0.022576,0.018588
1,0.020659,0.018013,-0.019889,0.016766,0.018541,0.018656,0.019635,0.001037,0.019217,0.000190,0.020182,0.018018,0.013323,0.018588
2,0.019958,0.019877,-0.019889,0.019822,0.021765,0.018383,0.019760,0.017104,0.020624,0.002329,0.020182,0.018018,0.022576,0.016898
3,0.020659,0.018013,-0.019889,0.016766,0.018541,0.020090,0.022867,0.027470,0.018905,0.000048,0.020182,0.018018,0.013323,0.016898
4,0.020659,0.018013,-0.019889,0.016766,0.018541,0.021603,0.019495,0.041464,0.018691,0.000018,0.020182,0.018018,0.013323,0.016898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,0.019375,0.017703,0.019889,0.018733,0.020153,0.020279,0.015161,0.001037,0.019168,0.000043,0.020182,0.018018,0.019985,0.016898
2524,0.019725,0.023293,-0.019889,0.023782,0.019347,0.018008,0.019495,0.016643,0.019445,0.000125,0.015137,0.018018,0.013323,0.016898
2525,0.018674,0.019566,0.019889,0.022287,0.019347,0.017529,0.026098,0.016643,0.019882,0.000064,0.020182,0.018018,0.018505,0.018588
2526,0.020308,0.020188,-0.019889,0.019443,0.018541,0.020548,0.019495,0.016643,0.021215,0.000643,0.020182,0.031532,0.019985,0.013519


In [22]:
X_norm

array([  85.67889413, 3219.80907819,   50.27922036, 1104.20171125,
       1240.52690418, 2610.52082164, 4023.35271413, 1929.40521588,
       1609.69440491,  204.39639127,  198.19434906,  221.99549545,
         27.02021465,   59.17786748])

## refine data

In [23]:
regr = sm.OLS(y_data, X_data)
regr_fit = regr.fit()

### leverage & outlier

In [24]:
infl = regr_fit.get_influence()

In [25]:
# leverage
leviers = infl.hat_matrix_diag
# list(leviers)

In [26]:
# studentized residue
res_studs = infl.resid_studentized_external
# list(res_studs)

### leverage detection
the threshold value may be:
$$s_h=2\times \frac{p+1}{n}$$

In [27]:
n_data = X_data.shape[0]
p_data = X_data.shape[1]

In [28]:
s_h = 2 * (p_data + 1) / n_data
s_h

0.011867088607594937

In [29]:
atyp_levier = leviers > s_h

In [30]:
len(leviers[atyp_levier])

125

### outlier detection
Threshold:
$$s_t=t_{1-0.05/2}(n-p-2)$$

In [31]:
seuil_stud = scipy.stats.t.ppf(0.975, df=n_data-p_data-2)
seuil_stud

1.9609088074865402

In [32]:
atyp_stud = np.abs(res_studs) > seuil_stud
atyp_stud

array([False, False, False, ...,  True, False,  True])

In [33]:
idx_eliminate_lvg_outl = np.logical_not(np.logical_or(atyp_levier, atyp_stud))

In [34]:
X_data.index[idx_eliminate_lvg_outl]

Int64Index([   0,    1,    2,    3,    4,    5,    7,    8,    9,   10,
            ...
            2516, 2517, 2518, 2519, 2520, 2521, 2522, 2523, 2524, 2526],
           dtype='int64', length=2275)

In [35]:
X_data_refine = X_data[idx_eliminate_lvg_outl]
y_data_refine = y_data[idx_eliminate_lvg_outl]

### save refine data (not normalized)

In [36]:
X_data_refine_not_norm = X_data_not_norm[idx_eliminate_lvg_outl]
y_data_refine_not_norm = y_data[idx_eliminate_lvg_outl]

In [37]:
df_refine_not_norm = pd.concat([X_data_refine_not_norm, y_data_refine_not_norm], axis=1)

In [38]:
df_refine_not_norm.to_csv('./processed_data/train_refine_not_norm.csv', index=False)

## feature selection

### backward selection

In [39]:
regr = sm.OLS(y_data_refine, X_data_refine)
regr_fit = regr.fit()

In [40]:
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          2.743e+05
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                        0.00
Time:                        00:44:16   Log-Likelihood:                         -4650.7
No. Observations:                2275   AIC:                                      9329.
Df Residuals:                    2261   BIC:                                      9410.
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3156.4007     86.299     36.575      0.000    2987.168    3325.633
weight       -1600.4649     68.492    -23.367      0.000   -1734.779   -1466.151
gender           8.0024      4.661      1.717      0.086      -1.138      17.143
bmi           1685.4500     63.125     26.700      0.000    1561.660    1809.240
age            -26.6331     30.860     -0.863      0.388     -87.150      33.883
rh              30.6710     12.794      2.397      0.017       5.582      55.760
heart_rate     -34.9514     16.885     -2.070      0.039     -68.063      -1.840
stress_level    27.3591      4.439      6.163      0.000      18.654      36.065
skin_temp      525.0952     77.703      6.758      0.000     372.719     677.471
eda              5.0463      7.424      0.680      0.497      -9.512      19.604
TC             110.7185     13.049      8.485      0.000      85.129     136.308
TS             126.6281     11.194     11.312      0.000     104.676     148.580
Clo           -120.6911      6.882    -17.538      0.000    -134.186    -107.196
Act             -7.5744      9.552     -0.793      0.428     -26.307      11.158
==============================================================================
Omnibus:                        6.139   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.046   Jarque-Bera (JB):                6.120
Skew:                           0.111   Prob(JB):                       0.0469
Kurtosis:                       2.878   Cond. No.                         223.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
X_rm = X_data_refine

#### remove eda

In [43]:
X_rm = X_rm.drop(columns='eda')

In [44]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          2.955e+05
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                        0.00
Time:                        00:44:47   Log-Likelihood:                         -4650.9
No. Observations:                2275   AIC:                                      9328.
Df Residuals:                    2262   BIC:                                      9402.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3148.6309     85.528     36.814      0.000    2980.909    3316.352
weight       -1593.4875     67.711    -23.534      0.000   -1726.269   -1460.706
gender           8.3081      4.639      1.791      0.073      -0.789      17.405
bmi           1679.3536     62.478     26.879      0.000    1556.834    1801.873
age            -26.0837     30.846     -0.846      0.398     -86.572      34.405
rh              30.7029     12.792      2.400      0.016       5.617      55.788
heart_rate     -34.3945     16.863     -2.040      0.042     -67.463      -1.326
stress_level    27.4596      4.436      6.190      0.000      18.760      36.159
skin_temp      530.8685     77.228      6.874      0.000     379.424     682.313
TC             111.1805     13.030      8.533      0.000      85.629     136.732
TS             126.5300     11.192     11.305      0.000     104.582     148.478
Clo           -120.6773      6.881    -17.538      0.000    -134.171    -107.184
Act             -7.4955      9.551     -0.785      0.433     -26.224      11.233
==============================================================================
Omnibus:                        6.251   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.044   Jarque-Bera (JB):                6.231
Skew:                           0.112   Prob(JB):                       0.0444
Kurtosis:                       2.877   Cond. No.                         220.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### remove Act

In [45]:
X_rm = X_rm.drop(columns='Act')

In [46]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          3.202e+05
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                        0.00
Time:                        00:44:48   Log-Likelihood:                         -4651.2
No. Observations:                2275   AIC:                                      9326.
Df Residuals:                    2263   BIC:                                      9395.
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3141.5228     85.040     36.942      0.000    2974.758    3308.287
weight       -1594.3713     67.696    -23.552      0.000   -1727.123   -1461.620
gender           7.8070      4.594      1.699      0.089      -1.202      16.816
bmi           1678.6575     62.466     26.873      0.000    1556.161    1801.154
age            -25.8171     30.841     -0.837      0.403     -86.297      34.663
rh              30.6677     12.791      2.398      0.017       5.584      55.751
heart_rate     -36.5184     16.643     -2.194      0.028     -69.156      -3.881
stress_level    27.4211      4.436      6.182      0.000      18.723      36.120
skin_temp      534.7960     77.059      6.940      0.000     383.682     685.910
TC             111.4974     13.022      8.562      0.000      85.960     137.035
TS             126.3490     11.189     11.293      0.000     104.408     148.290
Clo           -121.0701      6.862    -17.644      0.000    -134.527    -107.614
==============================================================================
Omnibus:                        6.256   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.044   Jarque-Bera (JB):                6.233
Skew:                           0.112   Prob(JB):                       0.0443
Kurtosis:                       2.876   Cond. No.                         212.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### remove age

In [47]:
X_rm = X_rm.drop(columns='age')

In [48]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          3.493e+05
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                        0.00
Time:                        00:44:49   Log-Likelihood:                         -4651.6
No. Observations:                2275   AIC:                                      9325.
Df Residuals:                    2264   BIC:                                      9388.
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3122.5100     81.945     38.105      0.000    2961.814    3283.206
weight       -1598.1831     67.538    -23.664      0.000   -1730.625   -1465.741
gender           7.0308      4.499      1.563      0.118      -1.792      15.854
bmi           1674.7297     62.285     26.888      0.000    1552.587    1796.872
rh              31.2427     12.772      2.446      0.015       6.197      56.288
heart_rate     -36.1820     16.637     -2.175      0.030     -68.808      -3.556
stress_level    27.9472      4.391      6.365      0.000      19.337      36.557
skin_temp      536.7229     77.019      6.969      0.000     385.687     687.759
TC             109.1103     12.706      8.588      0.000      84.195     134.026
TS             126.3230     11.188     11.291      0.000     104.384     148.263
Clo           -121.1119      6.861    -17.651      0.000    -134.567    -107.657
==============================================================================
Omnibus:                        5.900   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.052   Jarque-Bera (JB):                5.859
Skew:                           0.107   Prob(JB):                       0.0534
Kurtosis:                       2.872   Cond. No.                         200.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### remove gender

In [49]:
X_rm = X_rm.drop(columns='gender')

In [50]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          3.840e+05
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                        0.00
Time:                        00:44:49   Log-Likelihood:                         -4652.8
No. Observations:                2275   AIC:                                      9326.
Df Residuals:                    2265   BIC:                                      9383.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3140.7663     81.134     38.711      0.000    2981.662    3299.871
weight       -1678.7101     43.668    -38.442      0.000   -1764.344   -1593.076
bmi           1742.2392     44.881     38.819      0.000    1654.227    1830.251
rh              32.8291     12.735      2.578      0.010       7.855      57.803
heart_rate     -40.8119     16.376     -2.492      0.013     -72.926      -8.698
stress_level    28.0364      4.392      6.384      0.000      19.424      36.648
skin_temp      529.2686     76.896      6.883      0.000     378.474     680.063
TC             111.1921     12.639      8.797      0.000      86.406     135.978
TS             126.8577     11.186     11.341      0.000     104.922     148.794
Clo           -120.1319      6.835    -17.577      0.000    -133.535    -106.729
==============================================================================
Omnibus:                        7.733   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.021   Jarque-Bera (JB):                7.746
Skew:                           0.131   Prob(JB):                       0.0208
Kurtosis:                       2.883   Cond. No.                         187.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
X_rm = X_data_refine

start to remove features

### save selected data (not normalized)

In [52]:
X_selected = X_data_refine_not_norm.drop(columns=['eda', 'heart_rate', 'age', 'Act'])
y_selected = y_data_refine_not_norm

In [53]:
df_selected_not_norm = pd.concat([X_selected, y_selected], axis=1)

In [54]:
df_selected_not_norm.to_csv('./processed_data/train_selected_not_norm.csv', index=False)